In [1]:
import pandas as pd
import spiceypy as spy
import numpy as np
import rebound as rb
from Utils import *
from astroquery.jplhorizons import Horizons

In [2]:
fireballs = pd.read_csv('datos/cneos_fireball_data_original.csv', comment='#').sort_values(by=['Calculated Total Impact Energy (kt)'], ascending=False)
fireballs

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
376,2013-02-15 03:20:33,54.8N,61.1E,23.3,18.6,12.8,-13.3,-2.4,3.750000e+14,440.000
178,2018-12-18 23:48:20,56.9N,172.4E,26.0,13.6,6.3,-3.0,-31.2,3.130000e+13,49.000
494,2009-10-08 02:57:00,4.2S,120.6E,19.1,19.2,14.0,-16.0,-6.0,2.000000e+13,33.000
448,2010-12-25 23:24:00,38.0N,158.0E,26.0,18.1,18.0,-2.0,-4.0,2.000000e+13,33.000
954,1994-02-01 22:38:09,2.7N,164.1E,NaN,NaN,NaN,NaN,NaN,1.820000e+13,30.000
...,...,...,...,...,...,...,...,...,...,...
641,2005-04-16 10:40:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+10,0.073
181,2018-11-15 08:02:44,42N,57W,NaN,NaN,NaN,NaN,NaN,2.000000e+10,0.073
757,2002-03-18 14:44:57,60.4S,120.5W,NaN,NaN,NaN,NaN,NaN,2.000000e+10,0.073
48,2022-03-30 18:19:18,45.9S,171.4W,74.0,NaN,NaN,NaN,NaN,2.000000e+10,0.073


Notemos que el bolido de Chelyabinks es el de mayor energia de impacto con indice 376

In [3]:
indice = 376
meteor = fireballs.loc[indice]
meteor

Peak Brightness Date/Time (UT)         2013-02-15 03:20:33
Latitude (deg.)                                      54.8N
Longitude (deg.)                                     61.1E
Altitude (km)                                         23.3
Velocity (km/s)                                       18.6
vx                                                    12.8
vy                                                   -13.3
vz                                                    -2.4
Total Radiated Energy (J)                375000000000000.0
Calculated Total Impact Energy (kt)                  440.0
Name: 376, dtype: object

In [4]:
def change_coord(x):
    #funcion para trasformar el formato de coordenadas terrestres que da CNEOS
    if x[-1] == 'N' or x[-1] == 'E':
        new = float(x[:-1])
    elif x[-1] == 'S' or x[-1] == 'W':
        new = -float(x[:-1])
    return new    

#Parametros que obtenemos de los datos:
date = meteor['Peak Brightness Date/Time (UT)']
lon = change_coord(meteor['Longitude (deg.)'])
lat = change_coord(meteor['Latitude (deg.)'])
alt = meteor['Altitude (km)']
vx = meteor['vx']
vy = meteor['vy']
vz = meteor['vz']

date, lon, lat, alt, vx, vy, vz

('2013-02-15 03:20:33', 61.1, 54.8, 23.3, 12.8, -13.3, -2.4)

In [5]:
path = 'datos/kernels/'
spy.furnsh([path + 'naif0012.tls', path + 'pck00010.tpc', path + 'earth_fixed.tf', path + 'earth_720101_230601.bpc', path + 'earth_latest_high_prec.bpc'])

In [54]:
r = Geo2Rec(lon, lat, alt) 
r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='IAU_EARTH') #return units base on "alt" units

v = np.array([vx, vy, vz])  

T_earth = 86400  
omega = np.array([0,0,(2*np.pi)/T_earth]) 

v_E = (v - spy.vcrss(omega, r))
#v_E, -v, mag(v_E), np.arccos((v@r_irtf)/(np.linalg.norm(v)*np.linalg.norm(r_irtf)))*180/np.pi

et = spy.utc2et(date)
mx = spy.pxform('IAU_EARTH', 'ECLIPJ2000', et)
v_eclip = spy.mxv(mx, v_E)

### Integrando por 4 periodos orbitales

In [55]:
rb.horizons.SSL_CONTEXT = 'unverified'

sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "WHFast"
sim.dt = 86400

for body in ["Sun", "Earth", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]:
    sim.add(body, hash=body, date=date)

sun = sim.particles[0]
r_earth = np.array(sim.particles['Earth'].xyz)
v_earth = np.array(sim.particles['Earth'].vxyz)

r_asteroid = r_eclip  + r_earth 
v_asteroid = v_eclip + v_earth #así si es 

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mercury'... 
Found: Mercury Barycenter (199) (chosen from query 'Mercury')
Searching NASA Horizons for 'Venus'... 
Found: Venus Barycenter (299) (chosen from query 'Venus')
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mars'... 
Found: Mars Barycenter (4) (chosen from query 'Mars')
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')


In [56]:
AU = 149597870 #km
day = 86400

print("Position ECLIPJ200: ", r_asteroid/AU)
print("Velocity ECLIPJ200: ", (v_asteroid/AU)*day)

Position ECLIPJ200:  [-8.23964364e-01  5.43791769e-01 -2.23039340e-05]
Velocity ECLIPJ200:  [-0.01908999 -0.00988929 -0.00345246]


In [41]:
deg = 180/np.pi
AU = 149597870 #km
a = 1.73*AU
mu = 1.98847e30*sim.G
periodo_orb = 2*np.pi*np.sqrt(a**3/mu)

t_end = 4*periodo_orb 
print("tiempo de integracion: ", t_end)

N = 10
times = np.linspace(0, t_end, N)

orbit_elements = np.zeros((N, 6))
asteroid_statev = np.zeros((N, 6))
for i,time in enumerate(times):
    sim.integrate(-time)
    sim.move_to_com()
    asteroid_statev[i] = sim.particles[-1].xyz + sim.particles[-1].vxyz
    o = sim.orbits(primary=sun)[-1]
    orbit_elements[i] = [o.a/AU, o.e, o.inc*deg, o.Omega*deg, o.omega*deg, o.f] 

osc = orbit_elements[-1]
print(f"Elementos orbitales osculantes ultimo t: {time} s")   
print(f"a={osc[0]}, e={osc[1]}, i={osc[2]}, Omega={osc[3]}, omega={osc[4]}, f={osc[5]}")  

tiempo de integracion:  287238006.36966944
Elementos orbitales osculantes ultimo t: 287238006.36966944 s
a=2.3385762251932456, e=0.6993721124445695, i=10.253801255745914, Omega=-31.65654354775234, omega=106.1260321132565, f=3.133402373466005


### Calculando elementos orbitales osculantes con spy

In [42]:
r_int = asteroid_statev[-1]
et = spy.utc2et(date)
state = [r_int[0], r_int[1], r_int[2], r_int[3], r_int[4], r_int[5]]
mu = 1.98847e30*sim.G

osc_spy = spy.oscelt(state, et, mu)
print(f"Elementos orbitales osculantes ultimo t calculados con SPY: {time} s")   
print(f"q={osc_spy[0]/AU}, e={osc_spy[1]}, i={osc_spy[2]*deg}, Omega={osc_spy[3]*deg}, omega={osc_spy[4]*deg}, f={osc_spy[5]}") 

Elementos orbitales osculantes ultimo t calculados con SPY: 287238006.36966944 s
q=0.704557392937663, e=0.6989529853226245, i=10.247448922396474, Omega=328.4407221134385, omega=106.07595575994935, f=3.10938743481394


### Comparando datos del profe: 

Projected impact site:
Lat. : 55.07815N
Lon. : 60.09285E
Alt. : 178.4583 m

La fecha de impacto es:
Julian Date = 2456338.639279

La dirección del bolido era:
Radiant:
Az.  : 105.8913
Alt. : 17.84583

Las coordenadas del asteroide en el momento del impacto relativas a ECLIPJ2000 son:
Position vector at impact time (Ecliptic J2000.0):

x , y, z (AU) = -8.2296109700e-01 5.4628609900e-01 2.6028354800e-05
vx, vy, vz (AU/day) = -1.7823720100e-02 -9.2681019800e-03 -3.4069700000e-03 
r (AU) = 9.8777197200e-01

In [6]:
from astropy.time import Time
import spiceypy as spy

jd = 2456338.639279

t = Time(jd, format='jd', scale='utc')
utc_string = t.utc.iso 
et = spy.utc2et(utc_string)

print(f"UTC time       : {utc_string}")
print(f"Ephemeris Time : {et} seconds since J2000.0")

r = Geo2Rec(lon=60.09285, lat=55.07815, alt=178.4583) 
r_eclip = Geo2Eclip(lon=60.09285, lat=55.07815, alt=178.4583, et=et, frame='IAU_EARTH') #return units base on "alt" units

v = np.array([vx, vy, vz])  

T_earth = 2 * np.pi / 86400 + 2 * np.pi / (365.25 * 86400)
#T_earth = 86164.0905
omega = np.array([0,0,T_earth]) 
print(f"Rotation Vector : {omega}")

v_E = (v - spy.vcrss(omega, r))
mx = spy.pxform('IAU_EARTH', 'ECLIPJ2000', et)
v_eclip = spy.mxv(mx, v_E)

UTC time       : 2013-02-15 03:20:33.706
Ephemeris Time : 414170500.8911254 seconds since J2000.0
Rotation Vector : [0.00000000e+00 0.00000000e+00 7.29211543e-05]


In [7]:
rb.horizons.SSL_CONTEXT = 'unverified'

sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "WHFast"
#sim.dt = 86400

for body in ["Sun", "Earth", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]:
    sim.add(body, hash=body, date=date)

sun = sim.particles[0]
r_earth = np.array(sim.particles['Earth'].xyz)
v_earth = np.array(sim.particles['Earth'].vxyz)

r_asteroid = r_eclip  + r_earth 
v_asteroid = v_eclip + v_earth #así si es 

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mercury'... 
Found: Mercury Barycenter (199) (chosen from query 'Mercury')
Searching NASA Horizons for 'Venus'... 
Found: Venus Barycenter (299) (chosen from query 'Venus')
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mars'... 
Found: Mars Barycenter (4) (chosen from query 'Mars')
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')


In [8]:
AU = 149597870 #km
day = 86400

print("Position ECLIPJ200: ", r_asteroid/AU)
print("Velocity ECLIPJ200: ", (v_asteroid/AU)*day)
print("Velocity mag: ", mag(v_E))

Position ECLIPJ200:  [-8.23964890e-01  5.43791877e-01 -2.12928445e-05]
Velocity ECLIPJ200:  [-0.0190926  -0.00988658 -0.00345363]
Velocity mag:  18.875651936295935


In [9]:
deg = 180/np.pi
sim.move_to_com()
asteroid_statev = sim.particles[-1].xyz + sim.particles[-1].vxyz
o = sim.orbits(primary=sun)[-1]
orbit_elements = [o.a/AU, o.e, o.inc*deg, o.Omega*deg, o.omega*deg, o.f] 

print(f"Elementos orbitales instantaneos: ")   
print(f"a={orbit_elements[0]}, e={orbit_elements[1]}, i={orbit_elements[2]}, Omega={orbit_elements[3]}, omega={orbit_elements[4]}, f={orbit_elements[5]}")  

Elementos orbitales instantaneos: 
a=2.372424677691459, e=0.7016088739435956, i=10.410394984702243, Omega=-33.568615547324185, omega=108.2220963395764, f=1.2526114089179856


In [10]:
sim.status()

---------------------------------
REBOUND version:     	4.4.7
REBOUND built on:    	Mar  9 2025 20:56:14
Number of particles: 	11
Selected integrator: 	whfast
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x2effa3ee5d0, m=1.9884754159566474e+30 x=-150006.54756420263 y=-375025.82207393023 z=-7188.193107249768 vx=0.010826047142184584 vy=-0.0014637237934469036 vz=-0.00023452407127716457>
<rebound.particle.Particle object at 0x2effa3ed350, m=6.045825576341311e+24 x=-123261957.83132082 y=81351076.8213421 z=-9560.004964008403 vx=-16.949008934334103 vy=-24.930108733811668 vz=0.0005425020829763973>
<rebound.particle.Particle object at 0x2effa3ee5d0, m=3.301109449002709e+23 x=19405571.82401994 y=41433965.470469266 z=1614591.7183279435 vx=-53.85233524712189 vy=22.537614300916402 vz=6.783406057485624>
<rebound.particle.Particle object at 0x2effa3ed350, m=4.867466257521635e+24 x=57681863.87072127 y

In [11]:
hora = 3600

#t_end = 4*periodo_orb
t_end = 24*hora
print("tiempo de integracion: ", t_end)

N = 10
times = np.linspace(0, t_end, N)

orbit_elements = np.zeros((N, 6))
asteroid_statev = np.zeros((N, 6))
for i,time in enumerate(times):
    sim.integrate(-time)
    sim.move_to_com()
    asteroid_statev[i] = sim.particles[-1].xyz + sim.particles[-1].vxyz
    o = sim.orbits(primary=sun)[-1]
    int_elements = [o.a/AU, o.e, o.inc*deg, o.Omega*deg, o.omega*deg, o.f] 
    orbit_elements[i] = int_elements 

    print(f"Elementos orbitales en tiempo: {time} s")   
    print(f"a={int_elements[0]}, e={int_elements[1]}, i={int_elements[2]}, Omega={int_elements[3]}, omega={int_elements[4]}, f={int_elements[5]}")  

tiempo de integracion:  86400
Elementos orbitales en tiempo: 0.0 s
a=2.372424677691459, e=0.7016088739435956, i=10.410394984702243, Omega=-33.568615547324185, omega=108.2220963395764, f=1.2526114089179856
Elementos orbitales en tiempo: 9600.0 s
a=2.992970858390127, e=0.7296201293361579, i=10.088934178252336, Omega=-37.11861365833563, omega=119.19140235871187, f=1.0589862943876156
Elementos orbitales en tiempo: 19200.0 s
a=-1.9689180369255646, e=1.9248972851004795, i=9.114910799652465, Omega=-63.10339586594164, omega=180.61219892299536, f=6.269290692409124
Elementos orbitales en tiempo: 28800.0 s
a=-1.8817124183634264, e=1.9883995240288335, i=9.115017749689661, Omega=-63.71326280867493, omega=181.52939965734566, f=6.252029673819759
Elementos orbitales en tiempo: 38400.0 s
a=-1.8774834493799804, e=1.9914468172353859, i=9.114280491133819, Omega=-63.737055832177525, omega=181.5774487731444, f=6.249892493919305
Elementos orbitales en tiempo: 48000.0 s
a=-0.8826597062034937, e=5.564626354824